# HELLO Everyone. What's up ya'll
## NOPE
1. stuff
2. more stuff

In [1]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
from scipy import stats
import os
os.chdir("C:/Users/sam purkiss/Documents/Code/IMDB")

In [3]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python2  %%py

In [7]:
#Download the data
movie_graph_data = pd.read_csv('https://github.com/sampurkiss/movie_ratings_imdb/blob/master/tv_show_database.csv')


ParserError: Error tokenizing data. C error: Expected 1 fields in line 75, saw 4


In [6]:
###############################################
#Get graph showing movie ratings by proportion of respective gender
#x-values are mens rating, y-values are womans ratings
###############################################

hover_text =[]
for i in range(0,len(movie_graph_data)):
    hover_text.append(('Movie: {movie}<br>'+
                       'Release date: {date}<br>+
                       'Genre: {genre}').format(
                               movie = movie_graph_data['name'].iloc[i],
                               date = movie_graph_data['year'].iloc[i],
                               genre=movie_graph_data['genre'].iloc[i]))


traces =[]
for i in range(1,11):
    trace0 =go.Scatter(x = movie_graph_data['male_rating_'+str(i)]/movie_graph_data['no_of_male_ratings'],
                   y = movie_graph_data['female_rating_'+str(i)]/movie_graph_data['no_of_female_ratings'],
                   mode = 'markers',
                   text = hover_text,
                   name = str(i)+' stars')
    traces.append(trace0)

dissecting_line = go.Scatter(x =[0,2],
                            y = [0,2],
                            mode = 'lines',
                            name = 'Intersecting Line',
                            marker = dict(color = 'rgba(106,114,119,0.5)'))
traces.append(dissecting_line)

layout = go.Layout(title = 'Movie Ratings by Gender Proportion',
                   hovermode = 'closest',
                   showlegend = True,
                   xaxis = dict(title = 'Proportion of All Male Ratings',
                                range=[0, 1]),
                   yaxis = dict(title = 'Proportion of All Female Ratings',
                                range=[0, 1]))
fig = go.Figure(data = traces, layout = layout)

py.iplot(fig, filename='movie-ratings-by-gender')
